<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Notebook: Upload subjects to Zooniverse</h1>
<h3 align="right"><a href="https://colab.research.google.com/github/ocean-data-factory-sweden/kso/blob/main/notebooks/03_Upload_clips_to_Zooniverse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></h3>
<h3 align="right">Written by the KSO Team</h3>

This notebook takes you through the process of:

* Connecting to a Zooniverse project

* Extracting clips from videos stored locally or in the cloud.

* Modify these clips with for example a color correction or video compression.

* Upload the clips to Zooniverse for the 1st part of the species identification workflow (when does X species appear on a video).

If you do not have a project with us yet, you can run the template project to get a taste of how it all works. Only the uploading of the clips to Zooniverse will not be possible if you are not a member of our template project on Zooniverse.

🔴 <span style="color:red">&nbsp;NOTE: If you want to run another project than the template project, you need to have a Zooniverse account and be a member of the corresponding project.  </span>

# Set up KSO requirements

### Install all the requirements

Installing the requirements in Google Colab takes ~4 mins and might automatically crash/restart the session. Please run this cell until you get the "Successful installation!" message.

In [ ]:
import sys
import os

# Check if notebook is running in colab
IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    # Clone kso repo and install requirements
    if not os.path.exists("kso"):
        print("Installing all dependencies...")
        !git clone https://github.com/ocean-data-factory-sweden/kso.git
        !pip install -r /content/kso/requirements_colab.txt

    # Enable external widgets and navigate to the kso tutorial folder
    try:
        from google.colab import output

        output.enable_custom_widget_manager()
        os.chdir("kso/notebooks")
    except ImportError:
        pass

# Prepare the dev settings if needed
try:
    if "kso_utils" not in sys.modules:
        sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../..")))
        import kso_utils

        print("Using development version...")
        # Enables testing changes in utils
        %load_ext autoreload
        %autoreload 2
except ImportError:
    print("Installing latest version from PyPI...")
    %pip install -q kso-utils

if IN_COLAB:

    def restart_runtime():
        os.kill(os.getpid(), 9)

    # Check if there are any issues with previously imported packages
    try:
        from kso_utils.project import ProjectProcessor
    except Exception as e:
        print(f"Error importing package: {e}")
        print("Restarting runtime to apply package changes...")
        restart_runtime()

# Avoid issues with widgets not displaying properly
!jupyter nbextension enable --user --py widgetsnbextension
!jupyter nbextension enable --user --py jupyter_bbox_widget
!jupyter nbextension enable --user --py ipysheet

# Load the clear output function to keep things clean
from IPython.display import clear_output

clear_output()
print("Successful installation... you're good to go!")

### Import Python packages

In [ ]:
# Import required modules for tut#3
import kso_utils.widgets as kso_widgets
import kso_utils.project_utils as p_utils
from kso_utils.project import ProjectProcessor

print("Packages loaded successfully")

### Specify GPU availability

In [ ]:
gpu_available = kso_widgets.gpu_select()

### Choose your project

In [ ]:
project_name = kso_widgets.choose_project()

### Initiate project's database

In [ ]:
# Save the name of the project
project = p_utils.find_project(project_name=project_name.value)

# Initiate pp
pp = ProjectProcessor(project)

### Specify to request (or not) the latest Zooniverse info

In [ ]:
latest_zoo_info = kso_widgets.request_latest_zoo_info()

### Connect and retrieve information from the Zooniverse project

In [ ]:
pp.connect_zoo_project(latest_zoo_info.result)

# Task 1: Upload clips to Zooniverse

### Specify movie of interest

In [ ]:
pp.choose_footage(preview_media=True)

### Check if movie is already in Zooniverse

In [ ]:
# Remember to query the newest zooniverse data to get the most up to date list of clips uploaded
pp.check_movies_uploaded_zoo()

## Create some clip examples (Optional)
Test different parameters (e.g. compression rate, color modifications) in randomly selected clip examples

In [ ]:
pp.generate_zoo_clips(
    is_example=True,
    use_gpu=gpu_available.result,
)

In [ ]:
pp.check_clip_size()

In [ ]:
kso_widgets.compare_clips(
    example_clips=pp.generated_clips.clip_path,
    modified_clips=pp.generated_clips.clip_example_original_path,
)

## Create the clips to upload to Zooniverse

In [ ]:
pp.generate_zoo_clips(
    is_example=False,
    use_gpu=gpu_available.result,
)

### Check the size of the clips

In [ ]:
pp.check_clip_size()

## Upload clips to Zooniverse

If you receive an error message related to file size, we recommend shortening the clip length or compressing the clips to achieve a suitable filesize.

Make sure your workflows in Zooniverse have different names to avoid issues while creating a new workflow

🔴 <span style="color:red">&nbsp;NOTE: If you run the template project without being a member of our template project, it is not possible to run this last cell.  </span>

In [ ]:
pp.upload_zoo_subjects("clip")

In [ ]:
# END OF TASK 1

# Task 2: Upload images (frames) to Zooniverse

## Select Zooniverse workflow id and version of interest

##### Note: Make sure your workflows in Zooniverse have different names to avoid issues while selecting the workflow id

### Choose the workflows and versions of interest

In [ ]:
pp.choose_zoo_workflows()

### Sample and process Zooniverse classifications from the workflows of interest

In [ ]:
pp.process_zoo_classifications()

## Aggregate classifications received from the workflows of interest

### Aggregate classifications based on threshold

In [ ]:
users = kso_widgets.choose_aggregation_users(pp.processed_zoo_classifications)

### Specify the aggregation parameters

In [ ]:
agg_params = kso_widgets.choose_agg_parameters(
    pp.workflow_widget.checks["Subject type: #0"],
)

### Aggregate classifications based on parameters

In [ ]:
pp.aggregate_zoo_classifications(agg_params, users.result)

🔴 <span style="color:red">&nbsp;NOTE: If the output from the cell above says that 0 classifications are aggregated, you can experiment with other agreement thresholds, or you need to wait for more annotations to be made in Zooniverse.   </span>

### Select species of interest

In [ ]:
pp.species_of_interest = kso_widgets.choose_species(
    pp.db_connection, pp.aggregated_zoo_classifications["label"].unique()
)

### Extract frames from videos that have species of interest (based on selected aggreement)

In [ ]:
# Get all available frames for the selected species from clips
pp.extract_zoo_frames()

In [ ]:
# Review the size of the frames
pp.check_frame_size()

## Get frames from movies (Manual)

In [ ]:
# Choose folder containing movies
input_folder = kso_widgets.choose_folder()

In [ ]:
# Choose output folder for frames
output_folder = kso_widgets.choose_folder()

In [ ]:
# Generate suitable frames for upload by modifying initial frames
pp.generate_custom_frames(
    input_path=input_folder.selected,
    output_path=output_folder.selected,
    skip_start=120,
    skip_end=120,
    num_frames=10,
    frames_skip=None,
)

### Modify the frames if needed

In [ ]:
pp.modify_zoo_frames()

In [ ]:
# Review the size of the modified clips
pp.check_frame_size()

### Preview the frames

In [ ]:
# Compare the original and modified clips
pp.compare_frames(df=pp.modified_frames)

## Upload frames to Zooniverse

Make sure your workflows in Zooniverse have different names to avoid issues while creating a new workflow

🔴 <span style="color:red">&nbsp;NOTE: If you run the template project without being a member of our template project, it is not possible to run this last cell.  </span>

In [ ]:
pp.upload_zoo_subjects("frame")

In [ ]:
# END OF TASK 2